In [1]:
import findspark
findspark.init()

import pyspark
import pandas as pd
import numpy as np
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.pandas import *

C:\Spark\spark-3.4.0-bin-hadoop3\python\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

In [3]:
df = spark.read.json(path= r'C:\\Users\\ADMIN\\PycharmProjects\\Bigdata-1\\Data\\Dataset\\log_content\\20220401.json')

In [4]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [5]:
df = df.select('_source.*')
df.show(10)

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|  KPLUS|HNH743103|402343C25D7D|          251|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|
|  KPLUS|HND083642|B84DEE849A0F|         1444|
|  KPLUS|DNFD74404|90324BB44C39|          691|
|  KPLUS|DTFD21200|B84DEED27709|         1436|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|
+-------+---------+------------+-------------+
only showing top 10 rows



In [6]:
df.groupBy('AppName').count().orderBy('count', ascending= True).show(10)

+-------+-------+
|AppName|  count|
+-------+-------+
|   FIMS|   1007|
|  SPORT|   2298|
|  RELAX|   6445|
|  KPLUS|  20158|
|  CHILD|  33723|
|    VOD|  88033|
|CHANNEL|1502855|
+-------+-------+



In [7]:
df = df.withColumn("Type",when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
    .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
            (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
    .when((col("AppName") == 'RELAX'), "Giải Trí")
    .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
    .when((col("AppName") == 'SPORT'), "Thể Thao")
    .otherwise("Error"))

In [8]:
df.show(10)

+-------+---------+------------+-------------+-----------+
|AppName| Contract|         Mac|TotalDuration|       Type|
+-------+---------+------------+-------------+-----------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|Truyền Hình|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|Truyền Hình|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|Truyền Hình|
|  KPLUS|HND141717|08674EE8D2C2|         1452|Truyền Hình|
|  KPLUS|HNH743103|402343C25D7D|          251|Truyền Hình|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|Truyền Hình|
|  KPLUS|HND083642|B84DEE849A0F|         1444|Truyền Hình|
|  KPLUS|DNFD74404|90324BB44C39|          691|Truyền Hình|
|  KPLUS|DTFD21200|B84DEED27709|         1436|Truyền Hình|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|Truyền Hình|
+-------+---------+------------+-------------+-----------+
only showing top 10 rows



In [9]:
df = df.select('Contract', 'Type', 'TotalDuration')
df.show(10)

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|HNH579912|Truyền Hình|          254|
|HUFD40665|Truyền Hình|         1457|
|HNH572635|Truyền Hình|         2318|
|HND141717|Truyền Hình|         1452|
|HNH743103|Truyền Hình|          251|
|HNH893773|Truyền Hình|          924|
|HND083642|Truyền Hình|         1444|
|DNFD74404|Truyền Hình|          691|
|DTFD21200|Truyền Hình|         1436|
|LDFD05747|Truyền Hình|         1434|
+---------+-----------+-------------+
only showing top 10 rows



In [10]:
df = df.filter(df['Contract'] != '0')

In [11]:
df.groupBy('Contract').pivot('Type').sum('TotalDuration').count()

1360621

In [12]:
df.groupBy('Contract', 'Type').pivot('Type').sum('TotalDuration').count()

1483420

In [13]:
result = df.groupBy('Contract', 'Type').pivot('Type').sum('TotalDuration')
result.show(10)

+---------+-----------+--------+-----------+---------+--------+-----------+
| Contract|       Type|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+-----------+--------+-----------+---------+--------+-----------+
|DNH014998|Phim Truyện|    null|       3365|     null|    null|       null|
|HND486882|Phim Truyện|    null|       5545|     null|    null|       null|
|HUFD07189|Truyền Hình|    null|       null|     null|    null|       2264|
|HDFD36288|Truyền Hình|    null|       null|     null|    null|      11904|
|CTFD04401|Truyền Hình|    null|       null|     null|    null|      55881|
|HNH954607|Phim Truyện|    null|      13115|     null|    null|       null|
|HNH855959|Truyền Hình|    null|       null|     null|    null|        327|
|SGH034683|Truyền Hình|    null|       null|     null|    null|      82195|
|NTFD35330|Truyền Hình|    null|       null|     null|    null|      19139|
|NTFD48198|Phim Truyện|    null|      55202|     null|    null|       null|
+---------+-

In [14]:
result = result.withColumn('Date', lit('2022-04-1'))
result.show(10)

+---------+-----------+--------+-----------+---------+--------+-----------+---------+
| Contract|       Type|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|     Date|
+---------+-----------+--------+-----------+---------+--------+-----------+---------+
|DNH014998|Phim Truyện|    null|       3365|     null|    null|       null|2022-04-1|
|HND486882|Phim Truyện|    null|       5545|     null|    null|       null|2022-04-1|
|HUFD07189|Truyền Hình|    null|       null|     null|    null|       2264|2022-04-1|
|HDFD36288|Truyền Hình|    null|       null|     null|    null|      11904|2022-04-1|
|CTFD04401|Truyền Hình|    null|       null|     null|    null|      55881|2022-04-1|
|HNH954607|Phim Truyện|    null|      13115|     null|    null|       null|2022-04-1|
|HNH855959|Truyền Hình|    null|       null|     null|    null|        327|2022-04-1|
|SGH034683|Truyền Hình|    null|       null|     null|    null|      82195|2022-04-1|
|NTFD35330|Truyền Hình|    null|       null|     null|

In [15]:
# save_path = r"C:\Users\ADMIN\PycharmProjects\Bigdata-1\Data\Dataset\Clean_data"
# result.repartition(1).write.option("option_name", "20220401").mode("overwrite").csv(save_path, header=True)

In [16]:
result_df = result.select("*").toPandas()
result_df

,Contract,Type,Giải Trí,Phim Truyện,Thiếu Nhi,Thể Thao,Truyền Hình,Date
0,DNH014998,Phim Truyện,NaN,3365.0,NaN,NaN,NaN,2022-04-1
1,HND486882,Phim Truyện,NaN,5545.0,NaN,NaN,NaN,2022-04-1
2,HUFD07189,Truyền Hình,NaN,NaN,NaN,NaN,2264.0,2022-04-1
3,HDFD36288,Truyền Hình,NaN,NaN,NaN,NaN,11904.0,2022-04-1
4,CTFD04401,Truyền Hình,NaN,NaN,NaN,NaN,55881.0,2022-04-1
...,...,...,...,...,...,...,...,...
1483415,DNFD64591,Truyền Hình,NaN,NaN,NaN,NaN,14.0,2022-04-1
1483416,BDFD10096,Truyền Hình,NaN,NaN,NaN,NaN,15287.0,2022-04-1
1483417,THFD16458,Truyền Hình,NaN,NaN,NaN,NaN,22671.0,2022-04-1
1483418,HNH766004,Truyền Hình,NaN,NaN,NaN,NaN,22468.0,2022-04-1


In [17]:
file_name = '20220401'
save_path = r"C:\\Users\\ADMIN\\PycharmProjects\\Bigdata-1\\Data\Dataset\\Clean_data\\"
result_df.to_csv(save_path + file_name + '.csv', index=False)

In [ ]:
df = spark.createDataFrame([(1, 4, 3),
                            (4, None, 6)], ['a', 'b', 'c'])
df.show()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

df.select([F.isnull(df[col]) for col in df.columns]).show(10)

In [ ]:
missing_column_names = array(*[when(col(c).isNotNull(),lit(c)) for c in df.columns[:3]])
df = df.withColumn("missing_columns", missing_column_names)
df.show()

In [ ]:
df.withColumn("audit", array(*[when(col(c).isNull(), lit(c)) for c in df.columns[:3]]))\
.withColumn("audit", expr("filter(audit, x -> x is not null)"))\
.show()